# Week 2 coding tasks
## J.Merritt 2022_09_14

**1. First, import the primary care physicians dataset (`primary_care_physicians.csv`) into a data frame named `physicians`.**

In [204]:
import pandas as pd

In [205]:
physicians = pd.read_csv('../data/primary_care_physicians.csv')

In [206]:
physicians.head()

,FIPS,state,county,primary_care_physicians
0,1001,Alabama,Autauga,26.0
1,1003,Alabama,Baldwin,153.0
2,1005,Alabama,Barbour,8.0
3,1007,Alabama,Bibb,12.0
4,1009,Alabama,Blount,12.0


**2. Filter `physicians` down to just the counties in Tennessee. Save the filtered dataframe back to `physicians`. Verify that the resulting dataframe has 95 rows.**

In [207]:
physicians = physicians.loc[physicians['state'] == 'Tennessee']

In [208]:
physicians.info

<bound method DataFrame.info of        FIPS      state      county  primary_care_physicians
2432  47001  Tennessee    Anderson                     39.0
2433  47003  Tennessee     Bedford                     15.0
2434  47005  Tennessee      Benton                      3.0
2435  47007  Tennessee     Bledsoe                      1.0
2436  47009  Tennessee      Blount                     90.0
...     ...        ...         ...                      ...
2522  47181  Tennessee       Wayne                      5.0
2523  47183  Tennessee     Weakley                     18.0
2524  47185  Tennessee       White                      9.0
2525  47187  Tennessee  Williamson                    338.0
2526  47189  Tennessee      Wilson                     43.0

[95 rows x 4 columns]>

**3. Look at the distribution of the number of primary care physicians. What do you notice?**
##### There is a massive variation in the number of pcp's between counties

In [209]:
physicians.describe()

,FIPS,primary_care_physicians
count,95.000000,95.000000
mean,47095.000000,51.042105
std,55.136195,129.311426
min,47001.000000,0.000000
25%,47048.000000,4.500000
50%,47095.000000,12.000000
75%,47142.000000,26.500000
max,47189.000000,806.000000


**4. Now, import the population by county dataset (`population_by_county.csv`) into a DataFrame named `population`.**

In [210]:
population = pd.read_csv('../data/population_by_county.csv')

In [211]:
population.head()

,FIPS,population,county,state,urban
0,17051,21565,Fayette County,ILLINOIS,Rural
1,17107,29003,Logan County,ILLINOIS,Rural
2,17165,23994,Saline County,ILLINOIS,Rural
3,17097,701473,Lake County,ILLINOIS,Urban
4,17127,14219,Massac County,ILLINOIS,Rural


**5. Merge the `physicians` DataFrame with the `population` DataFrame. Keep only the values for Tennessee. When you merge, be sure the include both the `population` and `urban` columns in the merged results. Save the result of the merge back to `physicians`.**

In [212]:
physicians = pd.merge(left = physicians, 
                      right = population[['FIPS', 'population', 'urban']], 
                      on = 'FIPS')

In [213]:
# alternative:
# population['county'] = population['county'].str.replace(' County', '')
# physicians = pd.merge(left = physicians, 
#                 right = population.loc[population['state'] 
#                                        == 'TENNESSEE'][['population', 'urban', 'county']], 
#                 on = 'county'
#                 )

**6. How many Tennessee counties are considered urban?**
##### 38

In [214]:
len(physicians.loc[physicians['urban'] == 'Urban'])

38

**7. The State Health Access Data Assistance Center (SHADAC) (https://www.shadac.org/) classifies counties into three groups based on the number of residents per primary care physician. First, counties with fewer than 1500 residents per primary care physician are considered to have an "adequate" supply. Counties with at least 1500 residents but fewer than 3500 residents per primary care physician are considered to have a "moderately inadequate" supply, and counties with at least 3500 residents per primary care physician are considered to have a "low inadequate" supply. How many counties in Tennessee are in each group?** 

Create a column ('res_per_pcp') with values calculated from the 'population' divided by 'primary_care_physicians'.
Create a column 'health_access' populated with NA's, replate with 'adequate', 'moderately_inadequate', or 'low_inadequate' based on the conditions above
Create a new data frame ('tn_health_access') that displays the number of counties in each health access descriptor. 

In [215]:
physicians['res_per_pcp'] = physicians['population'] / physicians['primary_care_physicians']
physicians['health_access'] = 'NA'
physicians.loc[physicians['res_per_pcp'] < 1500, 'health_access'] = 'adequate'
physicians.loc[(physicians['res_per_pcp'] >= 1500) & 
               (physicians['res_per_pcp'] < 3500), 'health_access'] = 'moderately_inadequate'
physicians.loc[(physicians['res_per_pcp'] >= 3500) | 
               (physicians['primary_care_physicians'] == 0), 'health_access'] = 'low_inadequate'

In [216]:
tn_health_access = pd.DataFrame()

In [217]:
tn_health_access['health_access'] = ['adequate', 'moderately_inadequate', 'low_inadequate']
tn_health_access['counties'] = [len(physicians.loc[physicians['health_access'] == 'adequate']),
                                len(physicians.loc[physicians['health_access'] == 'moderately_inadequate']),
                                len(physicians.loc[physicians['health_access'] == 'low_inadequate'])
                               ]

In [218]:
tn_health_access

,health_access,counties
0,adequate,14
1,moderately_inadequate,50
2,low_inadequate,31


**8. Does there appear to be any detectable relationship between whether a county is urban or rural and its supply of primary care physicians?**
##### Rural counties are ~twice as likely to have low inadequate access to a pcp than an urban county.

Append the 'tn_health_access' data frame with new columns ('rural' and 'urban') with values computed for the number of urban or rural counties meeting each health access condition.


In [219]:
tn_health_access['urban'] = [len(physicians.loc[(physicians['health_access'] == 'adequate') &
                                                (physicians['urban'] == 'Urban')]),
                             len(physicians.loc[(physicians['health_access'] == 'moderately_inadequate') &
                                                (physicians['urban'] == 'Urban')]),
                             len(physicians.loc[(physicians['health_access'] == 'low_inadequate') &
                                                (physicians['urban'] == 'Urban')])
                               ]

In [220]:
tn_health_access['rural'] = [len(physicians.loc[(physicians['health_access'] == 'adequate') &
                                                ~(physicians['urban'] == 'Urban')]),
                             len(physicians.loc[(physicians['health_access'] == 'moderately_inadequate') &
                                                ~(physicians['urban'] == 'Urban')]),
                             len(physicians.loc[(physicians['health_access'] == 'low_inadequate') &
                                                ~(physicians['urban'] == 'Urban')])
                               ]

Compute normalized frequency of rural and urban counties within each health access grouping

In [221]:
tn_health_access['rural_to_urban'] = tn_health_access['rural'] / tn_health_access['urban']

In [222]:
tn_health_access

,health_access,counties,urban,rural,rural_to_urban
0,adequate,14,9,5,0.555556
1,moderately_inadequate,50,19,31,1.631579
2,low_inadequate,31,10,21,2.100000
